In [3]:
all_data = []
for directory in directories:
    print(PATH+directory)
    files = os.listdir(PATH+directory)
    files_list = [file for file in files[:len(files)-4]]
    for file in files_list:
        if file.endswith('.json'):
            filepath = PATH+os.path.join(directory, file)
            with open(filepath, 'r') as f:
                json_data = json.load(f)
                trace = json_data['trace']
                for j in range(len(trace)):
                    timestamp_in_seconds = str2int(trace[j]['timestamp']) / 1000
                    dt_object = datetime.fromtimestamp(timestamp_in_seconds)
                    tp = convert_to_unix_timestamp(dt_object)
                    all_data.append((j, tp, trace[j]['position']['x'], trace[j]['position']['y'], trace[j]['event_name'], directory))
df_combined = pd.DataFrame(all_data, columns=['userId', 'timestamp', 'x', 'y', 'eventName', 'userType'])
df_combined = df_combined.sort_values(by='timestamp')
df_combined['x'] = replace_zero_with_next_non_zero(df_combined['x'].to_numpy())
df_combined['y'] = replace_zero_with_next_non_zero(df_combined['y'].to_numpy())
df_combined['timestamp'] = pd.to_datetime(df_combined['timestamp'])
df_combined['time_diff'] = df_combined['timestamp'].diff().dt.total_seconds() * 1000
df_combined = df_combined.drop(columns=['timestamp'])


/kaggle/input/mouse-data/ISIT_Dataset/hlisa_traces
/kaggle/input/mouse-data/ISIT_Dataset/gremlins
/kaggle/input/mouse-data/ISIT_Dataset/za_proxy
/kaggle/input/mouse-data/ISIT_Dataset/survey_desktop


In [5]:
df_combined

,userId,x,y,eventName,userType,time_diff
655966,0,715,203,mouseup,survey_desktop,NaN
655967,1,715,203,error,survey_desktop,0.000123
655968,2,715,203,mouseover,survey_desktop,0.000007
655969,3,715,204,mousemove,survey_desktop,0.000660
655970,4,715,207,mousemove,survey_desktop,0.000014
...,...,...,...,...,...,...
519576,482,0,0,keypress,gremlins,0.000007
519577,483,0,0,keyup,gremlins,0.000021
493583,498,0,0,scroll,gremlins,0.000011
443978,470,0,0,scroll,gremlins,0.000038


In [14]:
print(df_combined['userId'].dtype)


int64


In [17]:
df_combined['userId'] = df_combined['userId'].astype(str).str.strip()
df1 = df_combined[df_combined['userId'] == '498']
df1.head()

,userId,x,y,eventName,userType,time_diff
656464,498,311,841,wheel,survey_desktop,0.000017
797787,498,142,992,mousemove,survey_desktop,0.000016
983673,498,174,909,mouseout,survey_desktop,0.000000
756871,498,78,1492,mouseup,survey_desktop,0.000025
772364,498,89,1017,wheel,survey_desktop,0.000004


In [5]:
event_counts = df['userType'].value_counts()
print(event_counts)

userType
survey_desktop    651160
Name: count, dtype: int64


In [3]:
import numpy as np
import pandas as pd
df_combined['distance'] = np.sqrt((df_combined['x'].diff())**2 + (df_combined['y'].diff())**2)
df_combined['speed'] = df_combined['distance'] / df_combined['time_diff'].replace(0, np.nan)
df_combined['acceleration'] = df_combined['speed'].diff() / df_combined['time_diff'].replace(0, np.nan)
df_combined['angle'] = np.arctan2(df_combined['y'].diff(), df_combined['x'].diff())
df_combined['angle_diff'] = df_combined['angle'].diff()
event_counts = pd.get_dummies(df_combined['eventName'])
df_combined = pd.concat([df_combined, event_counts], axis=1)
df_combined['is_idle'] = (df_combined['time_diff'] > df_combined['time_diff'].quantile(0.95)).astype(int)


agg_features = df_combined.groupby(['userId', 'userType']).agg({

    'speed': ['mean', 'std'],
    'acceleration': ['mean', 'std'],
    'angle_diff': ['mean', 'std'], 

}).reset_index()


agg_features.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in agg_features.columns.values]

print(agg_features.head())
print(agg_features['userType_'].value_counts())

   userId_       userType_    speed_mean     speed_std  acceleration_mean  \
0        0        gremlins  1.900077e+05  2.052464e+06       2.775404e+10   
1        0    hlisa_traces  3.014495e+06  2.299769e+07       8.744494e+11   
2        0  survey_desktop  9.500261e+06  3.590284e+07       1.418609e+12   
3        0        za_proxy  3.811165e+04  6.825461e+04      -6.465702e+10   
4        1        gremlins  1.246443e+04  1.028785e+05      -1.715611e+11   

   acceleration_std  angle_diff_mean  angle_diff_std  
0      4.720935e+11         0.004262        0.660038  
1      8.267663e+12         0.446417        1.323637  
2      5.777258e+12        -0.395266        1.258296  
3      5.264951e+10         0.587989        1.241790  
4      2.103663e+12         0.058661        0.499313  
userType_
survey_desktop    64743
hlisa_traces       1166
za_proxy            853
gremlins            577
Name: count, dtype: int64


In [7]:
from imblearn.over_sampling import SMOTE
from collections import Counter
smt = SMOTE()


In [4]:
df_major = agg_features[agg_features.userType_ == 'survey_desktop']
df_minor = agg_features[agg_features.userType_ != 'survey_desktop']
df_major = df_major.sample(frac=1, random_state=42).reset_index(drop=True)
agg_features = pd.concat([df_major, df_minor], axis=0).reset_index(drop=True)
print(agg_features['userType_'].value_counts())
df_major.shape

userType_
survey_desktop    64743
hlisa_traces       1166
za_proxy            853
gremlins            577
Name: count, dtype: int64


(64743, 8)

In [9]:
print(agg_features.shape)
agg_features = agg_features[agg_features.userType_ != 'za_proxy']
print(agg_features.shape)

(15545, 8)
(14692, 8)


In [10]:
agg_features.shape

(14692, 8)

In [37]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler
agg_features.fillna(0, inplace=True)
X = agg_features.drop(['userId_', 'userType_'], axis=1)
y = agg_features['userType_']

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(X)
x = pd.DataFrame(scaled_features, columns=X.columns)
# print(X_scaled.head())


In [33]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder

label_encoder = LabelEncoder()
agg_features.fillna(0, inplace=True)
X = agg_features.drop(['userId_', 'userType_'], axis=1)
y = agg_features['userType_']
y = label_encoder.fit_transform(y)
print(X.head())
print(y)

     speed_mean      speed_std  acceleration_mean  acceleration_std  \
0  3.750000e+05  530330.085890      -4.504442e+11      7.771893e+11   
1  0.000000e+00       0.000000       0.000000e+00      0.000000e+00   
2  1.500000e+06       0.000000      -2.500000e+11      0.000000e+00   
3  2.805846e+05  671452.045765       2.629287e+10      1.090830e+11   
4  7.218565e+05  318115.311544       2.828947e+11      6.222673e+11   

   angle_diff_mean  angle_diff_std  
0        -1.201747        1.768812  
1         0.000000        0.000000  
2         0.000000        0.000000  
3         0.334215        1.183790  
4         0.630849        0.829763  
[2 2 2 ... 1 1 1]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.ensemble import BalancedRandomForestClassifier
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3, random_state=42,stratify=y)

X_train_sm, y_train_sm = smt.fit_resample(X_train, y_train)